![car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-15</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [4]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Start coding!

data = pd.read_csv("car_insurance.csv")

data["credit_score"].fillna(data["credit_score"].mean(), inplace=True)
data["annual_mileage"].fillna(data["annual_mileage"].mean(), inplace=True)

accurancy_dict = {}
for col in data:
    if col != "id" and col != "outcome":
        outcomes = pd.DataFrame()
        model = logit(f"outcome ~ {col}", data=data).fit()

        actual_response = data[f"{col}"]
        predicted_response = np.round(model.predict())

        outcomes["actual_response"] = actual_response
        outcomes["predicted_response"] = predicted_response

        conf_matrix = model.pred_table()
        TN = conf_matrix[0, 0]
        TP = conf_matrix[1, 1]
        FN = conf_matrix[1, 0]
        FP = conf_matrix[0, 1]

        accurancy = (TN + TP) / (TN + TP + FN + FP)

        accurancy_dict[f"{col}"] = accurancy

# max accurancy value
max_accurancy_value = max(accurancy_dict.values())

# max accurancy key name
for key, value in accurancy_dict.items():
    if value == max_accurancy_value:
        max_accurancy_key = key

print("Max accurancy value is for column:", max_accurancy_key, "| Value is equal to:", max_accurancy_value)

best_feature_df = pd.DataFrame({"best_feature": [max_accurancy_key], "best_accuracy": [max_accurancy_value]})

print(best_feature_df)


                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.201
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     2517.
Date:                Sun, 14 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:33:40   Log-Likelihood:                -5384.4
No. Observations:               10000   AIC:                         1.077e+04
Df Residuals:                    9998   BIC:                         1.079e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6155      0.007     84.169      0.0